In [2]:
import os
import sys
import geopandas as gpd
import pandas as pd
%matplotlib notebook
import matplotlib.pyplot as plt

In [3]:
data_folder = "/Users/lena/Dropbox/Hackathon/rv"

In [4]:
hazard_ids = ['1m_sea-level','4m_storm-surge','flashflooding','landslide_susceptibility']

administrative = gpd.read_file(os.path.join(
    data_folder, 'data','admin', 'stluc_administrative.shp'))

admin_buildings = gpd.read_file(os.path.join(
    data_folder, 'results', 'admin_buildings.shp'))                           
                               
buildings = gpd.read_file(os.path.join(
    data_folder, 'data','buildings', 'stluc_buildings.shp'))


In [5]:
def load_hazard(data_folder, hazard_id):  
    hazard_path = os.path.join(
        data_folder, 'data', 'hazards', '{}.shp'.format(hazard_id))
    hazards = gpd.read_file(hazard_path)
    hazards = hazards.to_crs({'init':'epsg:2006'})
    return hazards

In [6]:
hazard_ids = ['1m_sea-level','4m_storm-surge','flashflooding']


In [6]:
hazard_ids = ['landslide_susceptibility']

### Intersect admin and buildings 

In [21]:
admin_buildings = gpd.sjoin(buildings, administrative, how="inner", op='intersects')
admin_buildings['count'] = 1
print(admin_buildings)
admin_buildings = admin_buildings[[
    'id_left',
    'admin_name',
    'count',
    'geometry'
]]
admin_buildings = admin_buildings.rename(columns={
    'id_left':'sector_id'
})
admin_buildings.to_file(os.path.join(
    data_folder, 'results', 'admin_buildings.shp'))
admin_buildings


KeyboardInterrupt: 

In [ ]:
all_intersections = []
for hazard_id in hazard_ids:
    hazard_df = load_hazard(data_folder, hazard_id)
    print(hazard_df)
    print(admin_buildings)
    intersections_sector_haz = gpd.sjoin(admin_buildings, hazard_df, how="inner", op='intersects')
    intersections_sector_haz[hazard_id] = 1
    intersections_sector_haz['sector'] = 'housing'
    #intersections_sector_haz = intersections_sector_haz[[
    #    'sector',
    #    'id_left', 
    #    hazard_id
    #]]
    #intersections_sector_haz = intersections_sector_haz.rename(columns={
    #    'id_left': 'sector_id' 
    #})
    #intersections_sector_haz = intersections_sector_haz.drop_duplicates(subset='sector_id', keep='first')
    intersections_sector_haz = intersections_sector_haz.fillna(0)
    
    
    all_intersections = all_intersections.reset_index()
    all_intersections = all_intersections[[
        'sector_id',
        'admin_name',
        'count',
        'landslide_susceptibility'
    ]]
    all_intersections.append(intersections_sector_haz)

all_intersections = pd.concat(all_intersections, axis=0, sort=False)
all_intersections = all_intersections.fillna(0)
all_intersections.to_csv(os.path.join(
    data_folder, 'results', 'intersect_buildings_landslide.csv'))
all_intersections

          id  risk                                           geometry
0          0     1  POLYGON ((513996.2998480261 1559596.475909927,...
1          1     1  POLYGON ((513896.6974260251 1559570.485524303,...
2          2     1  POLYGON ((513891.7344902826 1559565.35459989, ...
3          3     1  POLYGON ((513822.2302732103 1559498.630330347,...
4          4     1  POLYGON ((515173.5885640739 1559463.855924186,...
...      ...   ...                                                ...
15811  15811     1  POLYGON ((512988.023631359 1515474.461936977, ...
15812  15812     1  POLYGON ((512993.040091946 1515469.375136637, ...
15813  15813     1  POLYGON ((512682.4634847931 1515682.590854768,...
15814  15814     1  POLYGON ((513942.4207310621 1515371.324340834,...
15815  15815     1  POLYGON ((514724.3272987094 1515880.483854323,...

[15816 rows x 3 columns]
       sector_id  admin_name  count  \
0              0  Vieux Fort      1   
1              1  Vieux Fort      1   
2              2 

In [ ]:
all_intersections = all_intersections.reset_index()
all_intersections = all_intersections[[
    'sector_id',
    'admin_name',
    'count',
    '1m_sea-level', 
    '4m_storm-surge', 
    'flashflooding'
]]
all_intersections
all_intersections.to_csv(os.path.join(
    data_folder, 'results', 'intersect_buildings_inundation.csv'))
all_intersections